In [1]:
import json
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns

# 设置中文字体支持
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

class ActionAnalyzer:
    """分析JSONL格式的动作类型数据"""
    
    def __init__(self, jsonl_path):
        """
        初始化分析器
        
        Parameters:
        -----------
        jsonl_path : str
            JSONL文件路径
        """
        self.jsonl_path = jsonl_path
        self.data = []
        self.stats = None
        
    def load_data(self):
        """加载JSONL数据"""
        with open(self.jsonl_path, 'r', encoding='utf-8') as f:
            for line in f:
                if line.strip():
                    self.data.append(json.loads(line))
        print(f"✓ 成功加载 {len(self.data)} 条数据")
        return self
    
    def extract_action_type(self, id_string):
        """
        从ID中提取动作类型
        
        Parameters:
        -----------
        id_string : str
            格式: 数据集名称/动作类型/时间戳
            
        Returns:
        --------
        str : 动作类型
        """
        parts = id_string.split('/')
        if len(parts) >= 2:
            return parts[1]  # 返回两个斜杠之间的内容
        return None
    
    def analyze(self):
        """分析数据并生成统计信息"""
        action_data = defaultdict(lambda: {'trajectories': set(), 'samples': 0})
        
        for item in self.data:
            id_string = item.get('id', '')
            action_type = self.extract_action_type(id_string)
            
            if action_type:
                # 统计trajectory（完整的ID作为唯一标识）
                action_data[action_type]['trajectories'].add(id_string)
                # 统计样本数
                action_data[action_type]['samples'] += 1
        
        # 转换为DataFrame
        stats_list = []
        for action_type, info in action_data.items():
            stats_list.append({
                '动作类型': action_type,
                'Trajectory数量': len(info['trajectories']),
                '样本数量': info['samples']
            })
        
        self.stats = pd.DataFrame(stats_list).sort_values('样本数量', ascending=False)
        return self
    
    def print_summary(self):
        """打印统计摘要"""
        if self.stats is None:
            print("请先运行 analyze() 方法")
            return
        
        print("\n" + "="*60)
        print("📊 动作类型统计分析")
        print("="*60)
        
        print(f"\n总计：")
        print(f"  • 动作类型数量: {len(self.stats)}")
        print(f"  • Trajectory总数: {self.stats['Trajectory数量'].sum()}")
        print(f"  • 样本总数: {self.stats['样本数量'].sum()}")
        
        print(f"\n详细统计：")
        print(self.stats.to_string(index=False))
        
        print("\n" + "="*60)
        
    def plot_statistics(self, figsize=(14, 6)):
        """绘制统计图表"""
        if self.stats is None:
            print("请先运行 analyze() 方法")
            return
        
        fig, axes = plt.subplots(1, 2, figsize=figsize)
        
        # 图1：Trajectory数量
        axes[0].barh(self.stats['动作类型'], self.stats['Trajectory数量'], 
                     color='steelblue', alpha=0.8)
        axes[0].set_xlabel('Trajectory数量', fontsize=11)
        axes[0].set_title('各动作类型的Trajectory数量', fontsize=12, fontweight='bold')
        axes[0].grid(axis='x', alpha=0.3)
        
        # 图2：样本数量
        axes[1].barh(self.stats['动作类型'], self.stats['样本数量'], 
                     color='coral', alpha=0.8)
        axes[1].set_xlabel('样本数量', fontsize=11)
        axes[1].set_title('各动作类型的样本数量', fontsize=12, fontweight='bold')
        axes[1].grid(axis='x', alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    def get_dataframe(self):
        """返回统计DataFrame"""
        return self.stats
    
    def export_to_csv(self, output_path='action_statistics.csv'):
        """导出统计结果到CSV"""
        if self.stats is None:
            print("请先运行 analyze() 方法")
            return
        
        self.stats.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"✓ 统计结果已保存到: {output_path}")


# ==================== 使用示例 ====================

# 1. 创建分析器并加载数据
analyzer = ActionAnalyzer('/home/runsheng/personal_3/qiancx/ProgressLM/data/h5_agilex_3rgb_converted.jsonl')  # 替换为你的文件路径
analyzer.load_data()

# 2. 执行分析
analyzer.analyze()

# 3. 打印统计摘要
analyzer.print_summary()

# # 4. 绘制可视化图表
# analyzer.plot_statistics()

# # 5. 获取DataFrame进行进一步分析
# df = analyzer.get_dataframe()
# display(df)  # 在Jupyter中显示表格

# 6. 导出结果（可选）
# analyzer.export_to_csv('action_statistics.csv')

✓ 成功加载 56952 条数据

📊 动作类型统计分析

总计：
  • 动作类型数量: 61
  • Trajectory总数: 2262
  • 样本总数: 56952

详细统计：
                             动作类型  Trajectory数量  样本数量
                       41_putplum            36  1398
                   28_packtable_2            42  1377
                      13_packbowl            40  1263
              27_carrotgreenplate            46  1230
                     17_packtable            36  1218
                6_applebowloven_2            34  1197
                    56_placeplate            34  1149
                 42_putkiwifruite            36  1146
              7_applegreenplate_2            46  1128
                   10_packplate_2            40  1110
                   30_takepumpkin            49  1098
                    31_unpackbowl            47  1095
48_putpotatogreenplatefromsteam_2            37  1089
        59_steamedbreadlittleoven            34  1086
                    26_cleanplate            45  1059
                     36_putpepper        

In [2]:
import os
import json
import pandas as pd
from collections import defaultdict

# ===================== 配置区 =====================
INPUT_JSONL = '/home/runsheng/personal_3/qiancx/ProgressLM/data/h5_agilex_3rgb_converted.jsonl'   # ← 替换为你的 JSONL 路径
OUTPUT_DIR  = '/home/runsheng/personal_3/qiancx/Sources/datasets/robomind/progrsslm/annotation/h5_agilex_3rgb'            # ← 输出目录（会自动创建）
TRAIN_JSONL = os.path.join(OUTPUT_DIR, 'h5_agilex_3rgb_train.jsonl')
TEST_JSONL  = os.path.join(OUTPUT_DIR, 'h5_agilex_3rgb_test.jsonl')
SAVE_CLASS_LISTS = True                   # 是否同时导出类别清单

# —— 测试集类别（10类），其余全部为训练 ——
TEST_CLASSES = {
    '41_putplum',
    '28_packtable_2',
    '13_packbowl',
    '27_carrotgreenplate',
    '17_packtable',
    '56_placeplate',
    '30_takepumpkin',
    '26_cleanplate',
    '50_pourtea',
    '53_stackcup',
}
# =================================================

def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)

def extract_action_type(id_string: str):
    """
    从ID中提取动作类型
    约定: ID 形如 数据集名/动作类型/时间戳（至少包含两段）
    """
    parts = id_string.split('/')
    if len(parts) >= 2:
        return parts[1]
    return None

def load_jsonl(jsonl_path: str):
    data = []
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line))
    print(f"✓ 成功加载 {len(data)} 条数据")
    return data

def split_by_classes(data, test_classes: set):
    """按照类别集合划分 train/test（类别互斥）"""
    train, test = [], []
    for item in data:
        act = extract_action_type(item.get('id', ''))
        if act in test_classes:
            test.append(item)
        else:
            train.append(item)
    return train, test

def dump_jsonl(data, out_path: str):
    with open(out_path, 'w', encoding='utf-8') as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

def summarize_split(data, title: str):
    """
    输出 split 的基础统计（样本/trajectory/按类别分布）
    与你之前一致：以完整 id 作为 trajectory 的唯一标识
    """
    action_data = defaultdict(lambda: {'trajectories': set(), 'samples': 0})
    for item in data:
        id_string = item.get('id', '')
        action_type = extract_action_type(id_string)
        if action_type:
            action_data[action_type]['trajectories'].add(id_string)
            action_data[action_type]['samples'] += 1

    rows = []
    for act, info in action_data.items():
        rows.append({
            '动作类型': act,
            'Trajectory数量': len(info['trajectories']),
            '样本数量': info['samples']
        })
    df = pd.DataFrame(rows).sort_values('样本数量', ascending=False)

    n_actions = len(df)
    n_traj = int(df['Trajectory数量'].sum()) if not df.empty else 0
    n_samples = int(df['样本数量'].sum()) if not df.empty else 0

    print("\n" + "="*60)
    print(f"📊 {title} 统计")
    print("="*60)
    print(f"  • 动作类型数量: {n_actions}")
    print(f"  • Trajectory总数: {n_traj}")
    print(f"  • 样本总数: {n_samples}")
    if not df.empty:
        print("\n详细统计：")
        print(df.to_string(index=False))
    else:
        print("\n该 split 为空。")
    print("="*60 + "\n")

    return {'n_actions': n_actions, 'n_traj': n_traj, 'n_samples': n_samples, 'df': df}

def main():
    ensure_dir(OUTPUT_DIR)

    # 1) 读入
    data = load_jsonl(INPUT_JSONL)

    # 2) 切分（类别互斥）
    train, test = split_by_classes(data, TEST_CLASSES)

    # 3) 写出
    dump_jsonl(train, TRAIN_JSONL)
    dump_jsonl(test, TEST_JSONL)
    print(f"✓ 已写出 Train: {len(train)} 条 -> {TRAIN_JSONL}")
    print(f"✓ 已写出 Test : {len(test)} 条 -> {TEST_JSONL}")

    # 4) 统计（分别对 Train/Test）
    train_stat = summarize_split(train, "Train")
    test_stat  = summarize_split(test,  "Test")

    # 5) 汇总占比（便于 sanity check）
    total_samples = train_stat['n_samples'] + test_stat['n_samples']
    total_traj    = train_stat['n_traj'] + test_stat['n_traj']
    total_actions = train_stat['n_actions'] + test_stat['n_actions']

    print("📌 整体占比（按样本/trajectory/类别数）")
    if total_samples > 0:
        print(f"  • Test 样本占比: {test_stat['n_samples']}/{total_samples} = {test_stat['n_samples']/total_samples:.2%}")
    if total_traj > 0:
        print(f"  • Test Trajectory占比: {test_stat['n_traj']}/{total_traj} = {test_stat['n_traj']/total_traj:.2%}")
    if total_actions > 0:
        print(f"  • Test 类别占比: {test_stat['n_actions']}/{total_actions} = {test_stat['n_actions']/total_actions:.2%}")

    # 6) 可选：导出类别清单
    if SAVE_CLASS_LISTS:
        with open(os.path.join(OUTPUT_DIR, 'test_classes.txt'), 'w', encoding='utf-8') as f:
            f.write('\n'.join(sorted(TEST_CLASSES)))
        all_actions = set()
        for item in data:
            act = extract_action_type(item.get('id', ''))
            if act:
                all_actions.add(act)
        train_actions = sorted(all_actions - TEST_CLASSES)
        with open(os.path.join(OUTPUT_DIR, 'train_classes.txt'), 'w', encoding='utf-8') as f:
            f.write('\n'.join(train_actions))
        print("✓ 已保存 test_classes.txt 与 train_classes.txt")

    print("\n✅ 切分完成。类别互斥：Test = 10 类；Train = 其余全部类别。")

if __name__ == '__main__':
    main()


✓ 成功加载 56952 条数据
✓ 已写出 Train: 45276 条 -> /home/runsheng/personal_3/qiancx/Sources/datasets/robomind/progrsslm/annotation/h5_agilex_3rgb/h5_agilex_3rgb_train.jsonl
✓ 已写出 Test : 11676 条 -> /home/runsheng/personal_3/qiancx/Sources/datasets/robomind/progrsslm/annotation/h5_agilex_3rgb/h5_agilex_3rgb_test.jsonl

📊 Train 统计
  • 动作类型数量: 51
  • Trajectory总数: 1869
  • 样本总数: 45276

详细统计：
                             动作类型  Trajectory数量  样本数量
                6_applebowloven_2            34  1197
                 42_putkiwifruite            36  1146
              7_applegreenplate_2            46  1128
                   10_packplate_2            40  1110
                    31_unpackbowl            47  1095
48_putpotatogreenplatefromsteam_2            37  1089
        59_steamedbreadlittleoven            34  1086
                     36_putpepper            45  1056
                    22_takepotato            45  1032
                     10_packplate            36  1017
             9_appleyello

In [ ]:
import os
import json
import pandas as pd
from collections import defaultdict

# ===================== 配置区 =====================
INPUT_JSONL = '/Users/cxqian/Codes/ProgressLM/data/train/text_demo/text_h5_agilex_3rgb_rl.jsonl'   # ← 替换为你的 JSONL 路径
OUTPUT_DIR  = '/Users/cxqian/Codes/ProgressLM/data/eval/text'            # ← 输出目录（会自动创建）
TRAIN_JSONL = os.path.join(OUTPUT_DIR, 'text_h5_agilex_3rgb_train.jsonl')
TEST_JSONL  = os.path.join(OUTPUT_DIR, 'text_h5_agilex_3rgb_test.jsonl')
SAVE_CLASS_LISTS = True                   # 是否同时导出类别清单

# —— 测试集类别（10类），其余全部为训练 ——
TEST_CLASSES = {
    '41_putplum',
    '28_packtable_2',
    '13_packbowl',
    '27_carrotgreenplate',
    '17_packtable',
    '56_placeplate',
    '30_takepumpkin',
    '26_cleanplate',
    '50_pourtea',
    '53_stackcup',
}
# =================================================

def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)

def extract_action_type(id_string: str):
    """
    从ID中提取动作类型
    约定: ID 形如 数据集名/动作类型/时间戳（至少包含两段）
    """
    parts = id_string.split('/')
    if len(parts) >= 2:
        return parts[1]
    return None

def load_jsonl(jsonl_path: str):
    data = []
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line))
    print(f"✓ 成功加载 {len(data)} 条数据")
    return data

def split_by_classes(data, test_classes: set):
    """按照类别集合划分 train/test（类别互斥）"""
    train, test = [], []
    for item in data:
        act = extract_action_type(item.get('id', ''))
        if act in test_classes:
            test.append(item)
        else:
            train.append(item)
    return train, test

def dump_jsonl(data, out_path: str):
    with open(out_path, 'w', encoding='utf-8') as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

def summarize_split(data, title: str):
    """
    输出 split 的基础统计（样本/trajectory/按类别分布）
    与你之前一致：以完整 id 作为 trajectory 的唯一标识
    """
    action_data = defaultdict(lambda: {'trajectories': set(), 'samples': 0})
    for item in data:
        id_string = item.get('id', '')
        action_type = extract_action_type(id_string)
        if action_type:
            action_data[action_type]['trajectories'].add(id_string)
            action_data[action_type]['samples'] += 1

    rows = []
    for act, info in action_data.items():
        rows.append({
            '动作类型': act,
            'Trajectory数量': len(info['trajectories']),
            '样本数量': info['samples']
        })
    df = pd.DataFrame(rows).sort_values('样本数量', ascending=False)

    n_actions = len(df)
    n_traj = int(df['Trajectory数量'].sum()) if not df.empty else 0
    n_samples = int(df['样本数量'].sum()) if not df.empty else 0

    print("\n" + "="*60)
    print(f"📊 {title} 统计")
    print("="*60)
    print(f"  • 动作类型数量: {n_actions}")
    print(f"  • Trajectory总数: {n_traj}")
    print(f"  • 样本总数: {n_samples}")
    if not df.empty:
        print("\n详细统计：")
        print(df.to_string(index=False))
    else:
        print("\n该 split 为空。")
    print("="*60 + "\n")

    return {'n_actions': n_actions, 'n_traj': n_traj, 'n_samples': n_samples, 'df': df}

def main():
    ensure_dir(OUTPUT_DIR)

    # 1) 读入
    data = load_jsonl(INPUT_JSONL)

    # 2) 切分（类别互斥）
    train, test = split_by_classes(data, TEST_CLASSES)

    # 3) 写出
    dump_jsonl(train, TRAIN_JSONL)
    dump_jsonl(test, TEST_JSONL)
    print(f"✓ 已写出 Train: {len(train)} 条 -> {TRAIN_JSONL}")
    print(f"✓ 已写出 Test : {len(test)} 条 -> {TEST_JSONL}")

    # 4) 统计（分别对 Train/Test）
    train_stat = summarize_split(train, "Train")
    test_stat  = summarize_split(test,  "Test")

    # 5) 汇总占比（便于 sanity check）
    total_samples = train_stat['n_samples'] + test_stat['n_samples']
    total_traj    = train_stat['n_traj'] + test_stat['n_traj']
    total_actions = train_stat['n_actions'] + test_stat['n_actions']

    print("📌 整体占比（按样本/trajectory/类别数）")
    if total_samples > 0:
        print(f"  • Test 样本占比: {test_stat['n_samples']}/{total_samples} = {test_stat['n_samples']/total_samples:.2%}")
    if total_traj > 0:
        print(f"  • Test Trajectory占比: {test_stat['n_traj']}/{total_traj} = {test_stat['n_traj']/total_traj:.2%}")
    if total_actions > 0:
        print(f"  • Test 类别占比: {test_stat['n_actions']}/{total_actions} = {test_stat['n_actions']/total_actions:.2%}")

    # 6) 可选：导出类别清单
    if SAVE_CLASS_LISTS:
        with open(os.path.join(OUTPUT_DIR, 'test_classes.txt'), 'w', encoding='utf-8') as f:
            f.write('\n'.join(sorted(TEST_CLASSES)))
        all_actions = set()
        for item in data:
            act = extract_action_type(item.get('id', ''))
            if act:
                all_actions.add(act)
        train_actions = sorted(all_actions - TEST_CLASSES)
        with open(os.path.join(OUTPUT_DIR, 'train_classes.txt'), 'w', encoding='utf-8') as f:
            f.write('\n'.join(train_actions))
        print("✓ 已保存 test_classes.txt 与 train_classes.txt")

    print("\n✅ 切分完成。类别互斥：Test = 10 类；Train = 其余全部类别。")

if __name__ == '__main__':
    main()


✓ 成功加载 1488 条数据
✓ 已写出 Train: 1157 条 -> /Users/cxqian/Codes/ProgressLM/data/eval/text/text_h5_agilex_3rgb_train.jsonl
✓ 已写出 Test : 331 条 -> /Users/cxqian/Codes/ProgressLM/data/eval/text/text_h5_agilex_3rgb_test.jsonl

📊 Train 统计
  • 动作类型数量: 51
  • Trajectory总数: 164
  • 样本总数: 1157

详细统计：
                             动作类型  Trajectory数量  样本数量
                   10_packplate_2             4    32
                    22_takepotato             4    31
                 3_eggplantoven_2             4    31
                     1_potatooven             4    30
                    31_unpackbowl             4    30
48_putpotatogreenplatefromsteam_2             3    30
               8_appleblueplate_2             4    28
              7_applegreenplate_2             4    28
        59_steamedbreadlittleoven             3    28
                6_applebowloven_2             3    28
             9_appleyellowplate_2             4    27
                 42_putkiwifruite             3    26
           